## Instrucciones:

Este archivo es una plantilla que deberás completar para la creación de un job que permita ejecutar tu experimento asignado. Es importante reemplazar el nombre genérico del archivo por un identificador único, que será el número asignado al alumno que lo creó. Cada vez que otro alumno utilice ese job, se recompensará al autor original.

El job debe ser ejecutable simplemente configurando correctamente las siguientes secciones:
- **Parámetros**
- **Input**
- **Output**

El archivo debe ser obligatoriamente un *Jupyter Notebook*, con el fin de facilitar su uso a los alumnos menos técnicos. Sin embargo, si prefieres migrar o adaptar el código a archivos planos para tus propios pipelines, podrás hacerlo, siempre que quede claro el origen del código.

El formato del nombre del job debe seguir la siguiente convención:
- `j<id_alumno>_<ref>.ipynb`

Cada alumno puede generar más de un job, y también puede modificar los jobs de otros, siempre que se respete el nombre original.

Para la entrega de la segunda competencia, debes incluir dos componentes:
1. Todos los jobs utilizados, con sus nombres originales.
2. Un archivo explicando la secuencia en la que se ejecutaron los jobs.

Los jobs se compartirán en las slides de los experimentos con el link al repo de github.
---


# “Trazas de información ocultas bajo el análisis de los valores SHAP”

## Autor: Nahuel Alba
## Fecha de última modificación: <ie: 06/11/2024>
## Descripción: Al modelo optimizado aplicado al dataset se le aplica el algoritmo Shap value y se utiliza la importancia de las feature para realizar selectivamente transformaciones en su distribución con el algoritmo Yeo-Johnson.

< Completar >

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [ ]:
# Semillas que se utilizaran 
semillas = random.sample(range(1, 1000), 100)
# Parametros necesarios para calcular la ganancia
ganancia_acierto = 273000
costo_estimulo = 7000
# Meses de entrenamiento y prueba
mes_train = 202105
mes_test = 202107
# función para evaluar la ganancia
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True


## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [ ]:
# Ingrese la ruta donde se encuentra el archivo de datos con clase ternaria 

data = pd.read_csv(r"c:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\datasets\competencia_2\competencia_02_ct.csv")


## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [ ]:
df_experimento_unido = df_ganancias_post.merge(df_ganancias_default, left_on='semilla_post', right_on='semilla_default')


#ingrese path para guardar el archivo csv

df_experimento_unido.to_csv(r"C:\Users\Admin\Documents\1_Notebook\1_Estudio\1_UBA_Maestria_DS\1_Especializacion\1_Segundo_Semestre\DMEyF\experimento 1\datasets\df_experimento_unido.csv", index=False)

#output de mejora efectiva

experimento_resultado = df_experimento_unido['mejora']>0
experimento_resultado.values

In [ ]:
## Procesos

### Paquetes necesarios

In [ ]:
# Ejemplo
%pip install catboost

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [ ]:
data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)
df_train = data[data['foto_mes']<=mes_train]
df_test = data[data['foto_mes']==mes_test]
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train.loc[X_train.index, 'clase_peso']
w_data_test = df_test.loc[X_test.index, 'clase_peso']


In [ ]:
train_data = lgb.Dataset(X_train,
                            label=Y_train,  # elegir la clase
                            weight=clase_peso)

test_data = lgb.Dataset(X_test, label= Y_test, weight=w_data_test)


In [ ]:
#lgb default 

df_resultados_default = pd.DataFrame()
df_ganancias_default = pd.DataFrame({'semilla_default': [],'ganancias_default': []})
dicc_trazas_shap = {}


for x in semillas:
    params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    'seed': x }
    model_default = lgb.train(params, train_data)    
    y_pred_default = model_default.predict(X_test)
    ganancia = lgb_gan_eval(y_pred_default, test_data)
    df_resultados_default[f'default_{x}'] = y_pred_default
    df_ganancias_default = pd.concat([df_ganancias_default, pd.DataFrame([{'semilla_default': x, 'ganancias_default': ganancia}])], ignore_index=True)
    explainer = shap.TreeExplainer(model_default)
    shap_values = explainer.shap_values(X_train)
    shap_df = pd.DataFrame(shap_values, columns = X_train.columns)
    importancia_shap = pd.DataFrame(np.abs(shap_values).mean(0), columns = ['valores_shap'])
    importancia_shap['variables'] = X_train.columns
    importancia_shap = importancia_shap.sort_values(by = 'valores_shap', ascending = False)
    df_trazas_shap = importancia_shap[importancia_shap['valores_shap'] == 0]
    lista = df_trazas_shap['variables'].to_list()
    dicc_trazas_shap[x] = lista


In [ ]:
explainer = shap.TreeExplainer(model_default)
shap_values = explainer.shap_values(X_train)
shap_df = pd.DataFrame(shap_values, columns = X_train.columns)
importancia_shap = pd.DataFrame(np.abs(shap_values).mean(0), columns = ['valores_shap'])
importancia_shap['variables'] = X_train.columns
importancia_shap = importancia_shap.sort_values(by = 'valores_shap', ascending = False)

In [ ]:
df_trazas_shap = importancia_shap[importancia_shap['valores_shap'] == 0]
lista = df_trazas_shap['variables'].to_list()

In [ ]:
for x in semillas:
    dicc_trazas_shap[x] = lista
    valores_skew = []
    for variable in lista:
        valores_skew = []
        asimetria = skew(X_train[variable])
        valores_skew.append(asimetria)  
    df_trazas_shap['skew'] = valores_skew
ejemplo = lgb.Dataset(X_train, label=Y_train, weight=clase_peso)
df_ganancias_default['semilla_default'] = df_ganancias_default['semilla_default'].astype(int)

In [ ]:
# asimetria = skew(df['variable'])
dicc_valores_skew = {}
dicc_train_sets = {}
dicc_test_sets = {}

#creación datasets train
for semilla in semillas:
    dicc_train_sets[semilla] = pd.DataFrame(df_train)
#creación datasets test
for semilla in semillas:
    dicc_test_sets[semilla] = pd.DataFrame(df_test)

In [ ]:
df_ganancias_post = pd.DataFrame({'semilla_post': list(dicc_ganancia_post.keys()), 'ganancias_post': list(dicc_ganancia_post.values())})
df_experimento_unido = df_ganancias_post.merge(df_ganancias_default, left_on='semilla_post', right_on='semilla_default')
df_experimento_unido['ganancias_post'] = df_experimento_unido['ganancias_post'].apply(lambda x: x[1])
df_experimento_unido['ganancias_default'] = df_experimento_unido['ganancias_default'].apply(lambda x: x[1])
df_experimento_unido['mejora'] = df_experimento_unido['ganancias_post'] - df_experimento_unido['ganancias_default']
experimento_resultado = df_experimento_unido['mejora']>0
experimento_resultado.values